In [1]:
display(HTML("<style>.rendered_html.text_cell_render {max-width:700px; }</style>"));
push!(LOAD_PATH, "../src");
include("../src/pose.jl")

HTML{String}("<style>.rendered_html.text_cell_render {max-width:700px; }</style>")

In [2]:
#nbx --fname="src/house_expo_data.jl"
using JLD2
using StatsBase: mean
using Geometry: bounding_box, Segment
using Fmt
using MyUtils: summarize_vars

fname = "../data/data_4.jld2"
d = load_object(fname)

# Environment
_segs   = env_segs = d[:env][:segs];
_boxes  = env_segs = d[:env][:clutter];
_bb     = bounding_box(_segs)
_center = mean(_bb);

# Poses
_xs   = d[:motion][:x];
_hds  = d[:motion][:hd];
_ps   = [Pose(x,hd) for (x,hd) in zip(_xs, _hds)];

# Controls
_dxs  = d[:motion][:dx]
_dhds = d[:motion][:dhd]
_us   = [Control(dx,dhd) for (dx,dhd) in zip(_dxs, _dhds)]

_T = length(_xs);

In [11]:
#nbx
println("Loading data from `$(fname)`, and\npopulating Main namespace with variables:\n")
summarize_vars(:(_segs, _boxes, _center, _xs, _hds, _ps, _dxs, _dhds, _us, _T))

Loading data from `../data/data_4.jld2`, and
populating Main namespace with variables:

_segs      Vector{Segment}
_boxes     Vector{Segment}
_center    Vector{Float64}
_xs        Vector{Vector{Float64}}
_hds       Vector{Float64}
_ps        Vector{Pose}
_dxs       Vector{Vector{Float64}}
_dhds      Vector{Float64}
_us        Vector{Control}
_T         Int64
